In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error

In [12]:
# Load the time series data
data = pd.read_excel('rea_holdings_share_prices.xls',header=1)

In [13]:
# Convert the Date column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Set the Date column as the index
data.set_index('Date', inplace=True)

# Resample data to monthly frequency, taking the mean of 'Close' for each month
#monthly_data = data.resample('M').mean()


In [14]:
# Defining different lag periods to experiment with
lag_periods = [1,3,7,14,30]  # Monthly lag features for different periods

In [15]:
# Initializing dictionaries to store results
model_results = {'Random Forest': [], 'Gradient Boosting': [], 'Linear Regression': []}

In [16]:
data.head()

,Open,High,Low,Close
Date,,,,
2015-01-02,702.5,702.5,702.5,702.5
2015-01-05,697.5,697.5,697.5,697.5
2015-01-06,682.5,682.5,682.5,682.5
2015-01-07,690.0,690.0,690.0,690.0
2015-01-08,705.0,705.0,705.0,705.0


In [17]:
for lag_period in lag_periods:
    # Feature Engineering: Create lag features for the 'Close' price
    for i in range(1, lag_period + 1):
        data[f'lag_{i}'] = data['Close'].shift(i)
    
    # Drop rows with NaN resulting from lagged features
    data.dropna(inplace=True)

    # Split data into features (X) and target variable (y)
    X = data.drop(['Close'], axis=1)
    y = data['Close']

    # Train-Test Split (using TimeSeriesSplit for sequential split)
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Initialize and train the Random Forest model
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_mse = mean_squared_error(y_test, rf_pred)
        model_results['Random Forest'].append(rf_mse)

        # Initialize and train the Gradient Boosting model
        gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
        gb_model.fit(X_train, y_train)
        gb_pred = gb_model.predict(X_test)
        gb_mse = mean_squared_error(y_test, gb_pred)
        model_results['Gradient Boosting'].append(gb_mse)

        # Initialize and train the Linear Regression model
        lr_model = LinearRegression()
        lr_model.fit(X_train, y_train)
        lr_pred = lr_model.predict(X_test)
        lr_mse = mean_squared_error(y_test, lr_pred)
        model_results['Linear Regression'].append(lr_mse)

In [18]:
# Create a DataFrame from the model_results dictionary
model_results_df = pd.DataFrame(model_results)

# Print the DataFrame
print(model_results_df)

    Random Forest  Gradient Boosting  Linear Regression
0      820.755309         763.557991       6.080167e-27
1       40.847557          27.665668       4.308232e-27
2      113.711634         106.467077       7.904217e-27
3   211403.471142      203396.061684       2.193724e-25
4        5.793295          31.406741       1.737190e-26
5      859.378649         766.136937       2.403785e-27
6       44.740209          27.847724       3.475036e-27
7      151.210883         142.477842       5.208200e-27
8   207100.785323      204875.192467       4.112214e-25
9        9.951767          46.902896       1.724454e-26
10     966.474652         767.901766       5.969813e-26
11      69.015385          28.694019       1.644222e-24
12     189.236078         168.553982       3.324616e-26
13  209963.378561      206336.200326       3.765628e-26
14      15.880001          33.408647       5.306112e-26
15     977.302352         772.246509       8.028766e-26
16      83.427366          29.366485       1.676

## Data Drift

In [19]:
from scipy.stats import ks_2samp

# Assuming `X_train` is training features
drift_metrics = {}
for column in X_train.columns:
    statistic, p_value = ks_2samp(X_train[column], X_test[column])
    drift_metrics[column] = {'statistic': statistic, 'p_value': p_value}

drift_df = pd.DataFrame(drift_metrics).T
drift_df.to_csv('data_drift_metrics.csv')

In [20]:
print(drift_df)

        statistic        p_value
Open     0.775330  3.936038e-185
High     0.775330  3.936038e-185
Low      0.775330  3.936038e-185
lag_1    0.775881  1.881823e-185
lag_2    0.776432  8.986675e-186
lag_3    0.776982  4.286657e-186
lag_4    0.777533  2.042379e-186
lag_5    0.778084  9.719649e-187
lag_6    0.778634  4.620189e-187
lag_7    0.779185  2.193626e-187
lag_8    0.779736  1.040297e-187
lag_9    0.780286  4.927684e-188
lag_10   0.780837  2.331403e-188
lag_11   0.781388  1.101741e-188
lag_12   0.781938  5.200290e-189
lag_13   0.782489  2.451660e-189
lag_14   0.783040  1.154452e-189
lag_15   0.783590  5.429662e-190
lag_16   0.784141  2.550643e-190
lag_17   0.784692  1.196754e-190
lag_18   0.785242  5.608370e-191
lag_19   0.785793  2.625087e-191
lag_20   0.786344  1.227225e-191
lag_21   0.786894  5.730288e-192
lag_22   0.787445  2.672385e-192
lag_23   0.787996  1.244774e-192
lag_24   0.788546  5.790934e-193
lag_25   0.789097  2.690743e-193
lag_26   0.789648  1.248704e-193
lag_27   0

## Save Model Predictions and Results

In [25]:
# Calculate metrics
rf_metrics = {
    'MSE': mean_squared_error(y_test, rf_pred),
    'MAE': mean_absolute_error(y_test, rf_pred),
    'RMSE': mean_squared_error(y_test, rf_pred, squared=False),
    'R2': r2_score(y_test, rf_pred),
    'MedianAE': median_absolute_error(y_test, rf_pred)
}

gb_metrics = {
    'MSE': mean_squared_error(y_test, gb_pred),
    'MAE': mean_absolute_error(y_test, gb_pred),
    'RMSE': mean_squared_error(y_test, gb_pred, squared=False),
    'R2': r2_score(y_test, gb_pred),
    'MedianAE': median_absolute_error(y_test, gb_pred)
}

lr_metrics = {
    'MSE': mean_squared_error(y_test, lr_pred),
    'MAE': mean_absolute_error(y_test, lr_pred),
    'RMSE': mean_squared_error(y_test, lr_pred, squared=False),
    'R2': r2_score(y_test, lr_pred),
    'MedianAE': median_absolute_error(y_test, lr_pred)
}

# Create DataFrames for metrics
metrics_df = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting', 'Linear Regression'],
    'MSE': [rf_metrics['MSE'], gb_metrics['MSE'], lr_metrics['MSE']],
    'MAE': [rf_metrics['MAE'], gb_metrics['MAE'], lr_metrics['MAE']],
    'RMSE': [rf_metrics['RMSE'], gb_metrics['RMSE'], lr_metrics['RMSE']],
    'R2': [rf_metrics['R2'], gb_metrics['R2'], lr_metrics['R2']],
    'MedianAE': [rf_metrics['MedianAE'], gb_metrics['MedianAE'], lr_metrics['MedianAE']]
})

# Save metrics to CSV
metrics_df.to_csv('model_metrics.csv', index=False)

# Store predictions in DataFrame
results_df = pd.DataFrame({
    'Date': X_test.index,
    'Actual': y_test,
    'RF_Predicted': rf_pred,
    'GB_Predicted': gb_pred,
    'LR_Predicted': lr_pred
})

# Save predictions to CSV
results_df.to_csv('model_predictions.csv', index=False)

## Prepare Data for Tableau

In [24]:
combined_metrics_df = metrics_df.join(drift_df, how='outer')
combined_metrics_df.to_csv('combined_metrics.csv', index=False)